<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/aes_encrypt_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pynacl

In [ ]:
#!/usr/bin/env python3
"""
e2e_encrypt.py — Single-file end-to-end encryption using PyNaCl (libsodium).

Subcommands:
  - gen:     Generate a Curve25519 keypair (Base64-encoded files).
  - encrypt: Encrypt bytes for a recipient using their .pub key (SealedBox).
  - decrypt: Decrypt with your .secret key.

Install:
  pip install pynacl

Examples:
  python e2e_encrypt.py gen --out mykey
  python e2e_encrypt.py encrypt --pub recipient.pub --in plain.txt --out cipher.bin
  python e2e_encrypt.py decrypt --secret mykey.secret --in cipher.bin --out plain.txt
  echo "hello" | python e2e_encrypt.py encrypt --pub recipient.pub --out msg.enc
  python e2e_encrypt.py decrypt --secret mykey.secret --in msg.enc
"""

import argparse
import os
import sys
from typing import Optional

from nacl.public import PrivateKey, PublicKey, SealedBox
from nacl.encoding import Base64Encoder

B64 = Base64Encoder


def read_all_bytes(path: Optional[str]) -> bytes:
    if path is None or path == "-":
        return sys.stdin.buffer.read()
    with open(path, "rb") as f:
        return f.read()


def write_all_bytes(path: Optional[str], data: bytes):
    if path is None or path == "-":
        sys.stdout.buffer.write(data)
        sys.stdout.buffer.flush()
        return
    with open(path, "wb") as f:
        f.write(data)


def write_text(path: str, text: str):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)


def chmod_secret(path: str):
    # Best-effort: restrict secret key file on POSIX
    try:
        if os.name == "posix":
            os.chmod(path, 0o600)
    except Exception:
        pass


def load_public_key(path: str) -> PublicKey:
    raw = open(path, "r", encoding="utf-8").read().strip()
    try:
        return PublicKey(raw.encode("utf-8"), encoder=B64)
    except Exception as e:
        raise ValueError(f"Failed to parse public key (expect Base64): {e}")


def load_private_key(path: str) -> PrivateKey:
    raw = open(path, "r", encoding="utf-8").read().strip()
    try:
        return PrivateKey(raw.encode("utf-8"), encoder=B64)
    except Exception as e:
        raise ValueError(f"Failed to parse private key (expect Base64): {e}")


def cmd_gen(args: argparse.Namespace):
    sk = PrivateKey.generate()
    pk = sk.public_key

    secret_b64 = sk.encode(encoder=B64).decode("utf-8")
    pub_b64 = pk.encode(encoder=B64).decode("utf-8")

    secret_path = f"{args.out}.secret"
    pub_path = f"{args.out}.pub"

    if not args.overwrite:
        for p in (secret_path, pub_path):
            if os.path.exists(p):
                raise SystemExit(f"Refusing to overwrite existing file: {p}. Use --overwrite to allow.")

    write_text(secret_path, secret_b64 + "\n")
    chmod_secret(secret_path)
    write_text(pub_path, pub_b64 + "\n")

    print(f"Wrote private key: {secret_path}")
    print(f"Wrote public  key: {pub_path}")


def cmd_encrypt(args: argparse.Namespace):
    pk = load_public_key(args.pub)
    sealed = SealedBox(pk)

    plaintext = read_all_bytes(args.infile)
    ciphertext = sealed.encrypt(plaintext)  # bytes

    write_all_bytes(args.outfile, ciphertext)
    if args.outfile not in (None, "-"):
        print(f"Encrypted {len(plaintext)} bytes -> {len(ciphertext)} bytes to {args.outfile}")


def cmd_decrypt(args: argparse.Namespace):
    sk = load_private_key(args.secret)
    sealed = SealedBox(sk)

    ciphertext = read_all_bytes(args.infile)
    try:
        plaintext = sealed.decrypt(ciphertext)
    except Exception as e:
        raise SystemExit(f"Decryption failed: {e}")

    write_all_bytes(args.outfile, plaintext)
    if args.outfile not in (None, "-"):
        print(f"Decrypted {len(ciphertext)} bytes -> {len(plaintext)} bytes to {args.outfile}")


def build_parser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(
        description="Single-file end-to-end encryption (Curve25519 SealedBox)."
    )
    sub = p.add_subparsers(dest="cmd", required=True)

    pg = sub.add_parser("gen", help="Generate keypair (Base64-encoded files)")
    pg.add_argument("--out", required=True, help="Output prefix (writes <out>.secret and <out>.pub)")
    pg.add_argument("--overwrite", action="store_true", help="Allow overwriting existing files")
    pg.set_defaults(func=cmd_gen)

    pe = sub.add_parser("encrypt", help="Encrypt for recipient public key")
    pe.add_argument("--pub", required=True, help="Recipient public key file (.pub, Base64)")
    pe.add_argument("--in", dest="infile", default="-", help="Input file (default: stdin). Use - for stdin.")
    pe.add_argument("--out", dest="outfile", default="-", help="Output file (default: stdout). Use - for stdout.")
    pe.set_defaults(func=cmd_encrypt)

    pd = sub.add_parser("decrypt", help="Decrypt with your private key")
    pd.add_argument("--secret", required=True, help="Your private key file (.secret, Base64)")
    pd.add_argument("--in", dest="infile", default="-", help="Input file (default: stdin). Use - for stdin.")
    pd.add_argument("--out", dest="outfile", default="-", help="Output file (default: stdout). Use - for stdout.")
    pd.set_defaults(func=cmd_decrypt)

    return p


def main(argv=None):
    parser = build_parser()
    args = parser.parse_args(argv)
    args.func(args)


if __name__ == "__main__":
    main()

In [ ]:
!pip install cryptography

In [ ]:
#!/usr/bin/env python3
"""
aes_encrypt.py

End-to-end symmetric encryption using AES-GCM with PBKDF2-HMAC-SHA256.

Commands:
  encrypt --in  <infile>  --out <outfile>  --passphrase <pass>
  decrypt --in  <infile>  --out <outfile>  --passphrase <pass>

Use '-' for stdin/stdout.
"""

import argparse
import os
import sys

from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

# --- Configurable parameters ---
KDF_ITERATIONS = 390_000
SALT_SIZE     = 16    # bytes
NONCE_SIZE    = 12    # bytes (96-bit, recommended for AESGCM)
KEY_SIZE      = 32    # bytes (256-bit key)

def derive_key(passphrase: bytes, salt: bytes) -> bytes:
    """
    Derive a symmetric key from passphrase and salt via PBKDF2-HMAC-SHA256.
    """
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=KEY_SIZE,
        salt=salt,
        iterations=KDF_ITERATIONS,
    )
    return kdf.derive(passphrase)

def read_all(path: str) -> bytes:
    """
    Read all bytes from a file or stdin ('-').
    """
    if path == "-" or path is None:
        return sys.stdin.buffer.read()
    with open(path, "rb") as f:
        return f.read()

def write_all(path: str, data: bytes):
    """
    Write all bytes to a file or stdout ('-').
    """
    if path == "-" or path is None:
        sys.stdout.buffer.write(data)
        sys.stdout.buffer.flush()
    else:
        with open(path, "wb") as f:
            f.write(data)

def cmd_encrypt(args):
    # 1. Read plaintext
    plaintext = read_all(args.infile)
    passphrase = args.passphrase.encode("utf-8")

    # 2. Generate salt and derive key
    salt = os.urandom(SALT_SIZE)
    key  = derive_key(passphrase, salt)

    # 3. Encrypt with AESGCM
    aesgcm = AESGCM(key)
    nonce  = os.urandom(NONCE_SIZE)
    ct     = aesgcm.encrypt(nonce, plaintext, associated_data=None)

    # 4. Output: salt || nonce || ciphertext
    payload = salt + nonce + ct
    write_all(args.outfile, payload)
    if args.outfile not in ("-", None):
        print(f"Encrypted {len(plaintext)} → {len(payload)} bytes → {args.outfile}")

def cmd_decrypt(args):
    # 1. Read payload
    payload = read_all(args.infile)
    passphrase = args.passphrase.encode("utf-8")

    # 2. Split salt, nonce, ciphertext
    if len(payload) < (SALT_SIZE + NONCE_SIZE + 1):
        sys.exit("Payload too short or corrupted.")
    salt  = payload[:SALT_SIZE]
    nonce = payload[SALT_SIZE:SALT_SIZE+NONCE_SIZE]
    ct    = payload[SALT_SIZE+NONCE_SIZE:]

    # 3. Derive key & decrypt
    key = derive_key(passphrase, salt)
    aesgcm = AESGCM(key)
    try:
        plaintext = aesgcm.decrypt(nonce, ct, associated_data=None)
    except Exception as e:
        sys.exit(f"Decryption failed or tampered data: {e}")

    # 4. Write result
    write_all(args.outfile, plaintext)
    if args.outfile not in ("-", None):
        print(f"Decrypted {len(payload)} → {len(plaintext)} bytes → {args.outfile}")

def build_parser():
    p = argparse.ArgumentParser(description="AES-GCM file encrypt/decrypt")
    sub = p.add_subparsers(dest="cmd", required=True)

    pe = sub.add_parser("encrypt", help="Encrypt data")
    pe.add_argument("--in",  dest="infile",     default="-", help="Input file ('-'=stdin)")
    pe.add_argument("--out", dest="outfile",    default="-", help="Output file ('-'=stdout)")
    pe.add_argument("--passphrase", required=True, help="Encryption passphrase")
    pe.set_defaults(func=cmd_encrypt)

    pd = sub.add_parser("decrypt", help="Decrypt data")
    pd.add_argument("--in",  dest="infile",     default="-", help="Input file ('-'=stdin)")
    pd.add_argument("--out", dest="outfile",    default="-", help="Output file ('-'=stdout)")
    pd.add_argument("--passphrase", required=True, help="Decryption passphrase")
    pd.set_defaults(func=cmd_decrypt)

    return p

def main():
    parser = build_parser()
    args = parser.parse_args()
    args.func(args)

if __name__ == "__main__":
    main()